In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#let's add visualisation with the help TensorBoard
tbCallBack = keras.callbacks.TensorBoard(log_dir='Graph', histogram_freq=0,  
          write_graph=True, write_images=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                       callbacks=[tensorboard])

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using TensorFlow backend.


170500096/170498071 [==============================] - 36s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
  16/1563 [..............................] - ETA: 5:15 - loss: 2.3215 - acc: 0.0684

1562/1563 [============================>.] - ETA: 0s - loss: 1.8994 - acc: 0.3005

1563/1563 [==============================] - 299s 191ms/step - loss: 1.8991 - acc: 0.3005 - val_loss: 1.6321 - val_acc: 0.4104
Epoch 2/100
 449/1563 [=======>......................] - ETA: 3:24 - loss: 1.6735 - acc: 0.3861

1563/1563 [==============================] - 300s 192ms/step - loss: 1.6138 - acc: 0.4120 - val_loss: 1.4035 - val_acc: 0.4937
Epoch 3/100
  35/1563 [..............................] - ETA: 4:42 - loss: 1.5160 - acc: 0.4429

...

1563/1563 [==============================] - 297s 190ms/step - loss: 1.0748 - acc: 0.6518 - val_loss: 0.9277 - val_acc: 0.6952
Epoch 99/100
  34/1563 [..............................] - ETA: 4:37 - loss: 1.1129 - acc: 0.6324

1562/1563 [============================>.] - ETA: 0s - loss: 1.0873 - acc: 0.6474

1563/1563 [==============================] - 297s 190ms/step - loss: 1.0870 - acc: 0.6474 - val_loss: 0.8632 - val_acc: 0.7252
Epoch 100/100
 445/1563 [=======>......................] - ETA: 3:22 - loss: 1.0853 - acc: 0.6468

1563/1563 [==============================] - 296s 190ms/step - loss: 1.0809 - acc: 0.6476 - val_loss: 0.9109 - val_acc: 0.6900
Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 
 1440/10000 [===>..........................] - ETA: 12s

10000/10000 [==============================] - 15s 2ms/step
Test loss: 0.9108926626205445
Test accuracy: 0.69
